<a href="https://colab.research.google.com/github/DuqueGitWizard/Git-clone-DL-2/blob/main/tarea1_nlp_new_NEW_NEW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classificaton Fine-tuning

La siguiente tarea consiste en entrenar un modelo de HuggingFace (HF) para realizar la _task_ de _classification_. El dataset para entrenar dicho modelo está predefinido. Sin embargo, el modelo, el tokenizador y el trainer pueden ser totalmente personalizados. Es decir, que tendréis que realizar un trabajo de investigación, de prueba y error, para poder ir aprendiendo y ganando destreza con HF.

Recomendaciones:
- Durante este proceso, tendréis muchas dudas y encontraréis muchos errores. Tratad de resolverlas primero por vuestra cuenta, enteniendo la causa del error. Después con recursos online. Y, finalmente, siempre está el foro, que puede ser utilizado de forma participativa.
- No dejeis la tarea para el último día. Los modelos tardan en entrenar. Los problemas no se resuelven en la primera iteración.

Finalmente, se pide:
- Limpieza rigurosa en la presentación del notebook.
- El notebook se entrega con todas las celdas ejecutadas.
- Los comentarios (opcionales), mejor sobre el código con '#'.

Ánimo!

## Dataset

A continuación, descargarás un DatasectDict llamado _glue_. La target es la columna llamada _label_.

In [1]:
"""import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import torch
torch.cuda.empty_cache()  # Limpia la memoria caché de CUD
"""

'import os\nos.environ[\'CUDA_LAUNCH_BLOCKING\'] = "1"\nimport torch\ntorch.cuda.empty_cache()  # Limpia la memoria caché de CUD\n'

In [2]:
pip install transformers datasets torch accelerate scikit-learn nltk bokeh gensim spacy

In [3]:
pip install datasets

In [4]:
from datasets import load_dataset

ds = load_dataset("glue", "mnli")
ds

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9847
    })
})

In [5]:
print(ds['train'].column_names)

['premise', 'hypothesis', 'label', 'idx']


Lo primero que tendrás que hacer es construir un DatasetDict nuevo, llamado **ds_tarea**, que filtre el anterior DatasetDict para:
- quedarse con los registros que tengan el contenido de la columna _context_ con menos (estrictamente) de 20 caracteres.
- que solo tenga los Datasets de _train_ y _validation_matched_

In [6]:
ds_tarea = None

### BEGIN SOLUTION
# nos quedamos con los registros que tengan el contenido de la columna context con menos (estrictamente) de 20 caracteres.
# nos quedamos solo con los Datasets de train y validation_matched
ds_tarea = {
    'train': ds['train'].filter(lambda x: len(x['premise']) < 20),
    'validation_matched': ds['validation_matched'].filter(lambda x: len(x['premise']) < 20)
}
### END SOLUTION

ds_tarea

{'train': Dataset({
     features: ['premise', 'hypothesis', 'label', 'idx'],
     num_rows: 13635
 }),
 'validation_matched': Dataset({
     features: ['premise', 'hypothesis', 'label', 'idx'],
     num_rows: 413
 })}

In [7]:
# Celda de control

assert len(ds_tarea['train']) == 13635
assert len(ds_tarea['validation_matched']) == 413
assert set(ds_tarea.keys()) == {'train', 'validation_matched'}

## EDA

Si tenéis que realizar alguna exploración del datos, utilizad esta sección.

In [8]:
# Vemos las etiquetas únicas en la parte del train del dataset
unique_labels = set(ds_tarea['train']['label'])
num_labels = len(unique_labels)
num_labels # Sacamos el número de labels

3

In [9]:
# Vemos las etiquetas únicas en la parte del train del dataset
unique_labels = set(ds_tarea['validation_matched']['label'])
num_labels = len(unique_labels)
num_labels # Sacamos el número de labels

3

## Model y Tokenizer

Se pide guardar el modelo y el tokenizador en las variables _model_ y _tokenizer_.
Aunque no se utilicen hasta más adelante, declaradlos en esta sección.

In [10]:
tokenizer = None
model = None

### BEGIN SOLUTION
from transformers import GPT2ForSequenceClassification, GPT2Tokenizer, ElectraForSequenceClassification, ElectraTokenizer, AlbertForSequenceClassification, AlbertTokenizer, DistilBertForSequenceClassification, DistilBertTokenizer, XLNetForSequenceClassification, XLNetTokenizer, RobertaForSequenceClassification, RobertaTokenizer, AutoModelForSequenceClassification, AutoTokenizer, BertForSequenceClassification
"""Para guardar el modelo y el tokenizador, es necesario elegir un modelo pre-entrenado que se adapte a la clasificación de texto.
Usaremos el modelo bert-base-uncased de la libreria de HuggingFace Transformers"""
# Cargamos un modelo (AutoModelForSequenceClassification) para clasificar las secuencias y con from_pretrained("bert-base-uncased") cargamos un modelo preentrenado
# he usado bert-base-uncased pero podría haber usado otros como "roberta-base", "distilbert-base-uncased" o cualquiera de los que he probado más abajo que han obtenido un buen valor de accuracy
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)
# AutoTokenizer carga un  tokenizador correspondiente. Con from_pretrained("bert-base-uncased") cargamos el tokenizador correspondiente al modelo preentrenado.
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


"""A continuación muestro los diferentes modelos de preentrenamiento que se han probado. Las pruebas iniciales se han realizado con un num_train_epochs=1"""
"""
# Accuracy = 0.81 (num_train_epochs=1)
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
"""

""" OTRA OPCIÓN A PROBAR
model = AutoModelForSequenceClassification.from_pretrained("bert-large-uncased", num_labels=3)
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased")
"""
"""
# Accuracy = 0.799031 (num_train_epochs=1)
model = AutoModelForSequenceClassification.from_pretrained("bhadresh-savani/bert-base-uncased-emotion")
tokenizer = AutoTokenizer.from_pretrained("bhadresh-savani/bert-base-uncased-emotion")
"""

"""
# Accuracy = 0.34 (num_train_epochs=1)
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=3)
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
"""

""" OTRA OPCIÓN A PROBAR
model = RobertaForSequenceClassification.from_pretrained("roberta-large", num_labels=3)
tokenizer = RobertaTokenizer.from_pretrained("roberta-large")
"""

"""
# Accuracy = 0.34 (num_train_epochs=1)
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=3)
tokenizer = XLNetTokenizer.from_pretrained("xlnet-base-cased")
"""

"""
# Accuracy = 0.711864 (num_train_epochs=1)
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
"""

"""
# Accuracy = 0.460048 (num_train_epochs=1)
model = AlbertForSequenceClassification.from_pretrained("albert-base-v2", num_labels=3)
tokenizer = AlbertTokenizer.from_pretrained("albert-base-v2")
"""
"""
# Accuracy = 0.772397 (IMPORTANTE: tarda solo 5 min) (num_train_epochs=1)
model = ElectraForSequenceClassification.from_pretrained("google/electra-small-discriminator", num_labels=3)
tokenizer = ElectraTokenizer.from_pretrained("google/electra-small-discriminator")
"""
"""
# Accuracy = 0.736077 (num_train_epochs=1)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=3)
model.config.pad_token_id = tokenizer.pad_token_id
"""

# Por otro lado, con la función token_funct tokenizamos la columna 'premise'.
def token_funct(dataset):
    # He observamos que es necesario incluir dentro de la tokenización tanto la columna 'premise' como la columna 'hypothesis', ya que con esto le damos al modelo el contexto más correcto para entender la relación entre
    # ambos textos. En caso de incluir únicamente la columna 'premise' obserbamos que el valor del Accuracy no superaba el 0.35
    # Por lo tanto, para que el modelo se entrene correctamente utilizando todos los datos que son clave, es necesario incluir la información de ambas columnas.
    return tokenizer(dataset["premise"],dataset["hypothesis"], truncation=True, padding="max_length", max_length=256)

# Aplicamos la función token_funct a los subconjuntos 'train' y 'validation_matched' del dataset
ds_tarea = {
    'train': ds_tarea['train'].map(token_funct, batched=True),
    'validation_matched': ds_tarea['validation_matched'].map(token_funct, batched=True) # Con 'batched=True' indicamos que la función no se aplique uno
                                                                                        # por uno, sino por lotes (asi mejoramos la eficiencia)
}

#ds_tarea = ds_tarea.map(token_funct, batched=True)


### END SOLUTION

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Feature Engineering

Si tenéis que realizar alguna modificación de los datos (no siempre es necesaria, pero algunos modelos preentrenados lo piden), podéis utilizar esta sección.

Al finalizar la sección, bien si modificais el DatasectDict, bien si no lo modificáis, lo guardaréis en __ds_tarea_featured__.

In [11]:
# Celdas de libre uso

In [12]:
ds_tarea_featured = ds_tarea # Esta línea tiene sentido en caso de que no se modifique el dataset
#ds_tarea_featured['train'] = ds_tarea_featured['train'].rename_column('label','labels')
#ds_tarea_featured['validation_matched'] = ds_tarea_featured['validation_matched'].rename_column('label','labels')

### BEGIN SOLUTION



### END SOLUTION

ds_tarea_featured

{'train': Dataset({
     features: ['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 13635
 }),
 'validation_matched': Dataset({
     features: ['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 413
 })}

In [13]:
print(ds['train'].column_names)

['premise', 'hypothesis', 'label', 'idx']


In [14]:
# Celda de control

assert len(ds_tarea_featured['train']) == 13635
assert len(ds_tarea_featured['validation_matched']) == 413

## Fine-tuning

Para poder evaluar el modelo a lo largo del proceso y no esperar a tener toda la ejecución completa (que podría durar horas), se propone la creación de una métrica que muestre por pantalla la evolución del entrenamiento.

Esta métrica se declara en una función, llamada en este caso _compute_metrics_ y se le pide a los argumentos y al trainer que calculen la métrica al final de cada _epoch_ con el _evaluation_dataset_.

In [15]:
from datasets import load_metric

def compute_metrics(eval_pred):
    metric = load_metric("accuracy", trust_remote_code=True)
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return metric.compute(predictions=predictions, references=labels)

A continuación, de forma libre se pide entrenar un modelo de HuggingFace deseado. Se pide usar un Trainer de HuggingFace que tenga los siguientes argumentos como mínimo (puede haber más argumentos en todas las variables):

In [16]:
#pip install accelerate transformers

In [17]:
#pip install accelerate==0.21.0

In [18]:
from transformers import TrainingArguments, Trainer
import accelerate


### BEGIN SOLUTION
args = TrainingArguments(
    output_dir='./finetuned1', #
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=5e-5, # Tasa de aprendizaje
    logging_dir='./logs', # Guardar logs en este directorio
    #gradient_accumulation_steps=2,
    #per_device_train_batch_size=8,
    #per_device_eval_batch_size=16,
    logging_steps=10, # Cada 10  pasos de entrenamiento se va a registrar las métricas.
    num_train_epochs=10, # Defnimos 10 épocas de entrenamiento del modelo, es decir, se expone el modelo  a todos los datos de entrenamiento durante 10 veces.
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=ds_tarea_featured["train"],
    eval_dataset=ds_tarea_featured["validation_matched"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

### END SOLUTION

A continuación se entrena el modelo. Se pide no modificar esta celda, solo ejecutar.

In [ ]:
# Esta celda, celda tiene que estar ejecutada en la entrega

from time import time

start = time()

trainer.train()

end = time()
print(f">>>>>>>>>>>>> elapsed time: {(end-start)/60:.0f}m")

Epoch,Training Loss,Validation Loss,Accuracy
1,0.628900,0.553570,0.808717


<ipython-input-15-fcf706769f54>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy", trust_remote_code=True)


In [ ]:
# Esta celda tiene que estar ejecutada en la entrega
# Se espera un eval_accuracy superior a 0.75
# A mayor accuracy no hay mayor nota, con superar el umbral de 0.75 es suficiente

results = trainer.evaluate()
final_eval_accuracy = results.get("eval_accuracy")

print(f"Final Eval Accuracy: {final_eval_accuracy:.2f}")